## Predict

In [ ]:
from ultralytics import YOLO



Model = YOLO(
    model = './runs/detect/yolov8n_Infrared-Ocean-Target/weights/epoch100.pt' # Path to your yolo model
)

# Predict with model
PredictResult = Model.predict(
    source = './test.jpg' # Path to your test media
)

In [ ]:
import cv2
from ultralytics import YOLO


def Predicter(
    model: str,
    media: str
):
    # Load model
    Model = YOLO(model)

    def Predict(image):
        TickCount_Start = cv2.getTickCount()
        # Run YOLOv8 inference on the image
        Result = Model(image)
        # Visualize the results on the frame
        AnnotatedFrame = Result[0].plot()
        # Add info to the frame
        cv2.putText(
            img = AnnotatedFrame,
            text = f"FPS: {int(1 / ((cv2.getTickCount() - TickCount_Start) / cv2.getTickFrequency()))}",
            org = (10, 30), # topleft
            fontFace = cv2.FONT_HERSHEY_SIMPLEX,
            fontScale = 1,
            color = (0, 255, 0), # green
            thickness = 1
        )
        '''
        # Show the annotated frame
        cv2.imshow('img', AnnotatedFrame)
        '''
        return AnnotatedFrame

    # Predict video
    if media.endswith('.mp4') or media.endswith('.avi'):
        # Open the video file
        VideoCapture = cv2.VideoCapture(media)
        # Get the video's frame count
        VideoFrameCount = int(VideoCapture.get(cv2.CAP_PROP_FRAME_COUNT))
        # Get the video's frame size (width & height)
        VideoFrameSize = (int(VideoCapture.get(cv2.CAP_PROP_FRAME_WIDTH)), int(VideoCapture.get(cv2.CAP_PROP_FRAME_HEIGHT)))
        # Create a writer to save output
        VideoOutput = cv2.VideoWriter('./PredictResult.avi', cv2.VideoWriter_fourcc(*'XVID'), VideoFrameCount, VideoFrameSize)
        # Loop through the video frames
        while VideoCapture.isOpened():
            # Read a frame from the video
            IsRetrieving, frame = VideoCapture.read()
            if IsRetrieving:
                # Predict and then write the annotated frme into the video output
                VideoOutput.write(Predict(frame))
                # Break the loop if 'q' is pressed
                if cv2.waitKey(0) & 0xFF == ord("q"):
                    break
            else:
                # Break the loop if the end of the video is reached
                break
        # Release the video capture object and close the display window
        VideoCapture.release()
        cv2.destroyAllWindows()

    # Predict image
    if media.endswith('.jpg') or media.endswith('.png'):
        # Open the image file
        image = cv2.imread(media)
        # Predict and then save the annotated image
        cv2.imwrite('./PredictResult.png', Predict(image))
        cv2.waitKey(0)
        cv2.destroyAllWindows()


Predicter(
    model = './runs/detect/yolov8n_Infrared-Ocean-Target/weights/epoch100.pt', # Path to your yolo model,
    media = './test.jpg' # Path to your test media
)